In [1]:
import sys
sys.path.append('C:\Program Files\Anaconda3\Lib\site-packages')

In [1]:
import pandas as pd
from lxml import html
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
# from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
# stops = set(stopwords.words('russian'))

from stop_words import get_stop_words
stops = get_stop_words('russian')

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)


Корпус новостей для обучений векторных моеделей

In [2]:
data_rt = pd.read_csv('news_texts.csv')

In [3]:
data_rt.shape

(7217, 2)

In [4]:
data_rt.dropna(inplace=True)

In [5]:
data_rt.shape

(7212, 2)

In [6]:
data_rt.head()

,content,content_norm
0,Канцлер Германии Ангела Меркель в ходе брифинг...,канцлер германия ангел меркель ход брифинг пре...
1,Российские и белорусские войска успешно заверш...,российский белорусский войско успешно завершит...
2,"Дзюба, Шатов и Анюков оказались не нужны «Зени...",дзюба шат анюк оказаться нужный зенит российск...
3,"В Испанию без фанатов\nПожалуй, главной пятнич...",испания фанат пожалуй главный пятничный новост...
4,"Постпред России при ООН Виталий Чуркин, говоря...",постпред россия оон виталий чуркин говорить ве...


In [28]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=10000)
X = tfidf.fit_transform(data_rt['content_norm'])

In [29]:
X.shape

(7212, 10000)

##### SVD

In [30]:
%%time
svd = TruncatedSVD(50)
svd.fit(X)

CPU times: user 5.88 s, sys: 5.49 s, total: 11.4 s
Wall time: 2.69 s


##### NMF

In [31]:
%%time
nmf = NMF(50)
nmf.fit(X)

CPU times: user 50.1 s, sys: 40.7 s, total: 1min 30s
Wall time: 28.4 s


In [32]:
id2word = {i:w for i,w in enumerate(cv.get_feature_names())}
word2id = {w:i for i,w in id2word.items()}

In [33]:
id2vec_svd = nmf.components_.T

In [34]:
id2vec_nmf = svd.components_.T

In [35]:
def most_similar(word, id2vec):
    similar = [id2word[i] for i in cosine_distances(id2vec[word2id[word]].reshape(1, -1), id2vec).argsort()[0][:10]]
    return similar

In [36]:
most_similar('книга', id2vec_nmf)

['книга',
 'писатель',
 'творчество',
 'искусство',
 'произведение',
 'описывать',
 'знаменитый',
 'повесть',
 'поэт',
 'xx']

In [37]:
most_similar('книга', id2vec_svd)

['книга',
 'искусство',
 'писатель',
 'произведение',
 'знаменитый',
 'художник',
 'лауреат',
 'премия',
 'творчество',
 'описать']

In [16]:
from gensim.models import LdaModel
from gensim.models import fasttext

In [17]:
print(dir(fasttext))

['BaseWordEmbeddingsModel', 'FASTTEXT_FILEFORMAT_MAGIC', 'FAST_VERSION', 'FastText', 'FastTextKeyedVectors', 'FastTextTrainables', 'FastTextVocab', 'MAX_WORDS_IN_BATCH', 'REAL', 'Vocab', 'Word2VecTrainables', 'Word2VecVocab', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_compute_ngrams', '_ft_hash', 'call_on_class_only', 'deprecated', 'empty', 'logger', 'logging', 'np', 'np_sum', 'ones', 'struct', 'train_batch_cbow', 'train_batch_sg', 'train_cbow_pair', 'train_sg_pair', 'utils', 'vstack']


##### Word2vec

In [18]:
%%time
w2v = gensim.models.Word2Vec([text.split() for text in data_rt['content_norm']], size=50, sg=1)

CPU times: user 3min 51s, sys: 880 ms, total: 3min 51s
Wall time: 1min 21s


##### FastText

In [19]:
%%time
fast_text = fasttext.FastText([text.split() for text in data_rt['content_norm']], size=50, min_n=4, max_n=8)

CPU times: user 8min 34s, sys: 1.54 s, total: 8min 36s
Wall time: 3min 14s


In [20]:
fast_text.most_similar('книга')

/home/alapidus/miniconda3/envs/compling/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('мультфильм', 0.8800262212753296),
 ('кино', 0.8518549203872681),
 ('фильтр', 0.8429263234138489),
 ('фильм', 0.8424779176712036),
 ('мультсериал', 0.819197416305542),
 ('3234', 0.8115113377571106),
 ('кинолента', 0.8103165626525879),
 ('фил', 0.8021842837333679),
 ('кинематограф', 0.7988088130950928),
 ('сериал', 0.7928453683853149)]

In [21]:
w2v.most_similar('книга')

/home/alapidus/miniconda3/envs/compling/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('произведение', 0.8490647077560425),
 ('поэма', 0.834342896938324),
 ('литературный', 0.8266890048980713),
 ('картина', 0.8174421787261963),
 ('стихотворение', 0.7986379861831665),
 ('мультфильм', 0.7961488366127014),
 ('издательство', 0.7959349155426025),
 ('комикс', 0.7936421632766724),
 ('автобиография', 0.7908923625946045),
 ('титаник', 0.7905163764953613)]

##### Корпус парафразов

In [24]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [26]:
%%time
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

CPU times: user 27.4 s, sys: 0 ns, total: 27.4 s
Wall time: 27.5 s


In [59]:
data.head(10)

,label,text_1,text_2,text_1_norm,text_2_norm
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский разрешить стрелять поражение гражд...,полиция разрешить стрелять хулиган травматика
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия
5,1,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,пристав собрать отпечаток палец российский дол...,пристав снять отпечаток палец злостный неплате...
6,-1,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...,саратовский дебошир борт самолёт москва хургад...,саратовский дебошир отказываться возвращаться ...
7,0,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...,цик хотеть отказаться электронный система подс...,цик отказаться электронный средство подсчёт голос
8,-1,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...,суд петербург оставить дело гибель подросток п...,лондонский гайд-парк место митинг прежде парк
9,-1,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...,страна опека сократить добыча нефть 1 миллион ...,обама продлить полномочие наса сотрудничество ...


In [42]:
from sklearn.metrics.pairwise import cosine_similarity

######  SVD 

In [38]:
svd_text_1 = svd.transform(tfidf.transform(data['text_1_norm']))
svd_text_2 = svd.transform(tfidf.transform(data['text_2_norm']))

In [41]:
svd_text_1.shape

(7227, 50)

In [53]:
cosine_similarity(svd_text_1[0].reshape(1, -1), svd_text_2[0].reshape(1, -1))

array([[0.79504575]])

In [56]:
svd_sim = cosine_similarity(svd_text_1, svd_text_2).diagonal()

In [60]:
svd_sim[:10]

array([0.79504575, 0.81161594, 0.76032713, 0.63506641, 0.98274664,
       0.39290103, 0.47727611, 0.8832077 , 0.29858323, 0.13979391])

###### NMF

In [61]:
nmf_text_1 = nmf.transform(tfidf.transform(data['text_1_norm']))
nmf_text_2 = nmf.transform(tfidf.transform(data['text_2_norm']))


In [62]:
nmf_sim = cosine_similarity(nmf_text_1, nmf_text_2).diagonal()

In [63]:
nmf_sim[:10]

array([0.96485237, 0.92440748, 0.86394129, 0.53192501, 0.99169581,
       0.51535688, 0.33571858, 0.8416168 , 0.13748259, 0.0679591 ])

###### Word2vec

In [67]:
def get_embedding(text, model, dim):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [68]:
dim = 50
w2v_text_1 = np.zeros((len(data['text_1_norm']), dim))
w2v_text_2 = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    w2v_text_1[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    w2v_text_2[i] = get_embedding(text, w2v, dim)

/home/alapidus/miniconda3/envs/compling/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [70]:
w2v_text_1.shape

(7227, 50)

In [71]:
w2v_sim = cosine_similarity(w2v_text_1, w2v_text_2).diagonal()

In [72]:
w2v_sim[:10]

array([0.90944059, 0.91393835, 0.95933313, 0.72613995, 0.90710678,
       0.93119273, 0.78634835, 0.97720843, 0.64907291, 0.6467132 ])

###### FastText

In [73]:
dim = 50
fast_text_1 = np.zeros((len(data['text_1_norm']), dim))
fast_text_2 = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    fast_text_1[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    fast_text_2[i] = get_embedding(text, fast_text, dim)

/home/alapidus/miniconda3/envs/compling/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [74]:
fasttext_sim = cosine_similarity(fast_text_1, fast_text_2).diagonal()

In [75]:
fasttext_sim[:10]

array([0.80669522, 0.82055602, 0.83609194, 0.62733576, 0.69889539,
       0.77904897, 0.38122067, 0.88573169, 0.19446539, 0.14171555])

In [76]:
data['svd_sim'] = svd_sim

In [78]:
data['nmf_sim'] = nmf_sim
data['w2v_sim'] = w2v_sim
data['fasttext_sim'] = fasttext_sim

###### Adagram

In [88]:
import adagram

In [89]:
vm = adagram.VectorModel.load("out.pkl")

In [90]:
def get_embedding_adagram(text, model, window, dim):
    text = text.split()
    
    
    word2context = []
    for i in range(len(text)-1):
        left = max(0, i-window)
        word = text[i]
        left_context = text[left:i]
        right_context = text[i+1:i+window]
        context = left_context + right_context
        word2context.append((word, context))
    
    
    
    vectors = np.zeros((len(word2context), dim))
    
    for i,word in enumerate(word2context):
        word, context = word
        try:
            sense = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, sense)
            vectors[i] = v # просто умножаем вектор на частоту
        
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector
        

In [91]:
dim = 100
adagr_text_1 = np.zeros((len(data['text_1_norm']), dim))
adagr_text_2 = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    adagr_text_1[i] = get_embedding_adagram(text, vm, 5, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    adagr_text_2[i] = get_embedding_adagram(text, vm, 5, dim)

In [92]:
adagram_sim = cosine_similarity(adagr_text_1, adagr_text_2).diagonal()

In [93]:
data['adagram_sim'] = adagram_sim

###### Features Dataset

In [87]:
data.head()

,label,text_1,text_2,text_1_norm,text_2_norm,svd_sim,nmf_sim,w2v_sim,fasttext_sim,adagram_sim
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский разрешить стрелять поражение гражд...,полиция разрешить стрелять хулиган травматика,0.795046,0.964852,0.909441,0.806695,0.734838
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...,0.811616,0.924407,0.913938,0.820556,0.787086
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...,0.760327,0.863941,0.959333,0.836092,0.754871
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия,0.635066,0.531925,0.726140,0.627336,0.361842
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия,0.982747,0.991696,0.907107,0.698895,0.560512


In [94]:
data.to_csv('paraphrase_sim.csv', encoding = 'utf-8', index=False)

In [96]:
X = data[['svd_sim', 'nmf_sim', 'w2v_sim', 'fasttext_sim','adagram_sim']]

In [97]:
y = data['label']

In [100]:
train_X, valid_X, train_y, valid_y = train_test_split(X, y, random_state=14)
clf = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_leaf=10,
                             class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

             precision    recall  f1-score   support

         -1       0.67      0.65      0.66       642
          0       0.53      0.47      0.50       758
          1       0.43      0.55      0.48       407

avg / total       0.56      0.55      0.55      1807



In [109]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

In [110]:
kf = KFold(n_splits=5)
f1_scores = []
for train_index, test_index in kf.split(X):
    # print(train_index, test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_leaf=10,
                             class_weight='balanced')
    
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    f1_scores.append(f1_score(y_test, preds, average='micro'))
    
print(np.mean(f1_scores))

0.5404689227411735


In [118]:
kf = KFold(n_splits=5)
f1_scores = []
for train_index, test_index in kf.split(X):
    # print(train_index, test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = LogisticRegression(C=1000, class_weight='balanced',  multi_class='ovr')
    
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    f1_scores.append(f1_score(y_test, preds, average='micro'))
    
print(np.mean(f1_scores))

0.5491887416426176


И для логистической регрессии, и для RandomForest средняя F1-мера при микроусреднении - около 54%. Для логистической регрессии результат чуть лучше.
Попробуем добавить косинусную близость между TFIDF-векторами в качестве признака.

In [123]:
tfidf_text_1 = tfidf.transform(data['text_1_norm'])
tfidf_text_2 = tfidf.transform(data['text_2_norm'])

In [124]:
tfidf_sim = cosine_similarity(tfidf_text_1, tfidf_text_2).diagonal()

In [126]:
tfidf_sim[:20]

array([0.49287979, 0.67081215, 0.84998217, 0.15270967, 0.60388876,
       0.66088505, 0.39221454, 0.92201252, 0.        , 0.        ,
       0.77863067, 0.        , 0.37664076, 0.65858434, 0.79328721,
       0.63402837, 0.50096436, 0.82150569, 0.46494595, 0.65304967])

In [127]:
data['tfidf_sim'] = tfidf_sim

In [128]:
X = data[['svd_sim', 'nmf_sim', 'w2v_sim', 'fasttext_sim','adagram_sim', 'tfidf_sim']]

In [129]:
kf = KFold(n_splits=5)
f1_scores = []
for train_index, test_index in kf.split(X):
    # print(train_index, test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = LogisticRegression(C=1000, class_weight='balanced',  multi_class='ovr')
    
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    f1_scores.append(f1_score(y_test, preds, average='micro'))
    
print(np.mean(f1_scores))

0.5520967518078749


F1-мера поднялась до 55% :)

Попробуем добавить tfidf взвешивание для w2v и fasttext представлений.

In [147]:
tfidf.idf_

array([4.39057879, 5.35104074, 5.07961919, ..., 5.60697411, 5.91334832,
       6.35727971])

In [148]:
idf_dict = dict([(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

In [150]:
def tfidf_embedding(text, model, dim, idf):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total)*idf[word] # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [151]:
dim = 50
fasttfidf_text_1 = np.zeros((len(data['text_1_norm']), dim))
fasttfidf_text_2 = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    fasttfidf_text_1[i] = tfidf_embedding(text, fast_text, dim,idf_dict)
    
for i, text in enumerate(data['text_2_norm'].values):
    fasttfidf_text_2[i] = tfidf_embedding(text, fast_text, dim,idf_dict)

/home/alapidus/miniconda3/envs/compling/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [153]:
dim = 50
w2vtfidf_text_1 = np.zeros((len(data['text_1_norm']), dim))
w2vtfidf_text_2 = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    w2vtfidf_text_1[i] = tfidf_embedding(text, w2v, dim,idf_dict)
    
for i, text in enumerate(data['text_2_norm'].values):
    w2vtfidf_text_2[i] = tfidf_embedding(text, w2v, dim,idf_dict)

/home/alapidus/miniconda3/envs/compling/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [154]:
fasttfidf_sim = cosine_similarity(fasttfidf_text_1, fasttfidf_text_2).diagonal()

In [155]:
fasttfidf_sim[:10]

array([0.84091925, 0.89149951, 0.85495967, 0.66555334, 0.92271267,
       0.71993618, 0.60916878, 0.95228467, 0.33120287, 0.17634337])

In [156]:
fasttext_sim[:10]

array([0.80669522, 0.82055602, 0.83609194, 0.62733576, 0.69889539,
       0.77904897, 0.38122067, 0.88573169, 0.19446539, 0.14171555])

In [157]:
w2vtfidf_sim = cosine_similarity(w2vtfidf_text_1, w2vtfidf_text_2).diagonal()

In [158]:
data['fasttfidf_sim'] = fasttfidf_sim
data['w2vtfidf_sim'] = w2vtfidf_sim

In [171]:
X = data[['svd_sim', 'nmf_sim', 'w2v_sim', 'fasttext_sim','adagram_sim', 'tfidf_sim', 'fasttfidf_sim', 'w2vtfidf_sim']]

In [172]:
kf = KFold(n_splits=5)
f1_scores = []
for train_index, test_index in kf.split(X):
    # print(train_index, test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = LogisticRegression(C=1000, class_weight='balanced',  multi_class='ovr')
    
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    f1_scores.append(f1_score(y_test, preds, average='micro'))
    
print(np.mean(f1_scores))

0.5478070515489575


In [ ]:
F1-мера осталась на том же уровне.